In [0]:
"""           Notebook: Bronze – Portal da Transparência

Objetivo: Realizar a ingestão dos arquivos originais do Portal de Transparência
para a camada Bronze, preservando o formato bruto dos dados em enexo a pipe do case.

Nesta etapa não há transformações de negócio mantenho os dado brutos.
"""


In [0]:
df = spark.table("default.202510_cpgf")


In [0]:
from pyspark.sql import functions as F

tabelas_origem = [
    ("default.202510_afastamentos", "bronze_afastamentos"),
    ("default.202507_cadastro", "bronze_cadastro"),
    ("default.202507_remuneracao", "bronze_remuneracao"),
    ("default.202508_cpgf", "bronze_cpgf")
]

for origem, destino in tabelas_origem:

    df = spark.table(origem)

    df_bronze = (
        df.withColumn("ingest_ts", F.current_timestamp())
          .withColumn("source_table", F.lit(origem))
    )

    df_bronze.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(destino)


In [0]:
spark.table("bronze_cpgf").printSchema()


root
 |-- codOrgaoSuperior: integer (nullable = true)
 |-- json: string (nullable = true)
 |-- ingest_ts: timestamp (nullable = true)
 |-- source_table: string (nullable = true)



In [0]:
spark.table("default.bronze_cpgf") \
    .select(F.substring("json", 1, 200).alias("inicio_json")) \
    .show(truncate=False)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|inicio_json                                                                                                                                                                                             |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{"id":1,"codOrgaoSuperior":20000,"nomOrgaoSuperior":"Presidência da República","codOrgao":20415,"nomOrgao":"Empresa Brasil de Comunicação","codUnidadeGestora":115406,"nomUnidadeGestora":"EMPRESA BRAS|
|[{"id":792,"codOrgaoSuperior":22000,"nomOrgaoSuperior":"Ministério da Agricultura e Pecuária","codOrgao":22202,"nomOrgao":"Empresa Brasileira de Pesquisa Agropecuária","codUnidadeGestora"